In [ ]:
from project.models import Project, Emprise, Plan, PlanEmprise
from public_data.models import CommunesSybarval as Comm
from django.contrib.gis.db.models import Union
from django.contrib.gis.db.models.functions import Intersection, Area, Transform
from users.models import User
from django.conf import settings
from django.core.serializers import serialize
from prettytable import PrettyTable
from django.db.models import F
import random

default_user = User.objects.get(email__startswith="swann")

In [ ]:
# show available project to let you choose an id
x = PrettyTable(field_names=["id", "Nom du projet"], align="l")
x.add_rows([(p.id, p.name) for p in Project.objects.all().order_by("id")])
print(x)

In [ ]:
# get project and combined emprise
test_project = Project.objects.get(pk=2)
print(f"Project: {test_project.name}")
print("=========================")
geom = test_project.combined_emprise

test_filters = ["coveredby", "within", "contained", "crosses", "disjoint", "intersects", "overlaps"]
# contained : boss avec le rectangle englobant

communes = {_._commune: [] for _ in Comm.objects.all()}

for t in test_filters:
    for city_name in communes.keys():
        communes[city_name].append("")
    for city in Comm.objects.filter(**{f"mpoly__{t}": geom}):
        communes[city._commune][-1] = "OUI"

x = PrettyTable(field_names=["Commune"]+test_filters, align="l")
for name, row in communes.items():
    x.add_row([name]+row)
print(x)

In [ ]:
from rest_framework_gis.serializers import GeoFeatureModelSerializer, GeometryField
from rest_framework import serializers

class CustomSerializer(GeoFeatureModelSerializer):
    # intersection_area = serializers.FloatField()
    intersection = GeometryField()
    
    class Meta:
        model = Comm
        geo_field = "intersection"
        fields=('_commune', )

test_project = Project.objects.get(pk=4)
geom = test_project.combined_emprise
qs = Comm.objects.filter(mpoly__intersects=geom)
qs = qs.annotate(intersection=Intersection("mpoly", geom))
qs = qs.annotate(intersection_area=Area("intersection"))
qs = qs.annotate(area=Area("mpoly"))
for city in qs:
    print(city._commune, city.area /  1000000, city.intersection_area)
    
serializer = CustomSerializer(qs, many=True)
    
data = serializer.data
data

In [ ]:
from ipyleaflet import Map, GeoJSON
from rest_framework_gis.serializers import GeoFeatureModelSerializer
from django.contrib.gis.db.models.functions import Intersection, Area, Transform
from django.conf import settings
# from my django app
from public_data.models import CommunesSybarval as Comm
from project.models import Project, Emprise, Plan, PlanEmprise

class CustomSerializer(GeoFeatureModelSerializer):
    intersection = GeometryField()
    
    class Meta:
        model = Comm
        geo_field = "intersection"
        fields=('_commune', )

# get the data from the database
geom = Project.objects.get(pk=4).combined_emprise
qs = Comm.objects.filter(mpoly__intersects=geom).annotate(intersection=Intersection("mpoly", geom))
qs = qs.annotate(intersection_area=Area("intersection")).annotate(area=Area("mpoly"))
    
serializer = CustomSerializer(qs, many=True)

# Start map renderint
m = Map(center=(44.6586, -1.164), zoom=11)

geo_json = GeoJSON(
    data=serializer.data,
    style={'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1},
    hover_style={'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5},
    style_callback={'color': 'black',
                    'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),}
)
m.add_layer(geo_json)
m

In [ ]:
# lambert 93 - EPSG 2154
mios = Comm.objects.get(_commune="Mios")

def get_km_sq(mpoly): 
    """ 
    Returns the area in acres. 
    """ 
    # Convert our geographic polygons (in WGS84)
    # into a local projection for France (here lambert 93 - EPSG 2154) 
    mpoly.transform(2154)
    # we are in m² and we want km²
    return mpoly.area / (1000 * 1000)

print(get_km_sq(mios.mpoly))

In [ ]:
test_project = Project.objects.get(pk=4)
geom = test_project.combined_emprise
qs = Comm.objects.filter(mpoly__intersects=geom)
qs = qs.annotate(mpoly_2=Transform("mpoly", 2154))
# qs = qs.annotate(intersection=Intersection("mpoly", geom))
# qs = qs.annotate(intersection2=Transform("intersection", 2154))
qs = qs.annotate(intersection=Transform(Intersection("mpoly", geom), 2154))
qs = qs.annotate(intersection_area=Area("intersection"))
qs = qs.annotate(area=Area("mpoly_2"))

x = PrettyTable()
x.field_names = ["Commune", "Aire (km²)", "Aire (km²) intersection COBAS", ]
x.align = "r"
x.align["Commune"] = "l"
for city in qs:
    x.add_row([city._commune, f"{city.area.sq_km:.2f}", f"{city.intersection_area.sq_km:.2f}"])
print(x)

In [ ]:
qs = qs.filter(intersection_area__gt=F("area") * 0.95 )

x = PrettyTable()
x.field_names = ["Commune", "Aire (km²)", "Aire (km²) intersection COBAS", ]
x.align = "r"
x.align["Commune"] = "l"
for city in qs:
    x.add_row([city._commune, f"{city.area.sq_km:.2f}", f"{city.intersection_area.sq_km:.2f}"])
print(x)

In [ ]:
from django.db import connection
qs = Comm.objects.filter(code_insee__in=["33527", ])
qs = qs.aggregate(mpoly=Union("mpoly"))
connection.queries.pop()

In [ ]:
from django.contrib.gis.geos import Polygon as dj_Polygon

poly = dj_Polygon.from_bbox((43, -1, 44, 0))
poly.srid = 4326
poly_2 = dj_Polygon.from_bbox((43.25, -0.75, 43.75, -0.25))
poly_3 = dj_Polygon.from_bbox((43.4, -0.6, 43.6, -0.4))


clone = poly.transform(2154, clone=True)
clone.area / 10000

In [ ]:
from ipyleaflet import Map, Polygon

m = Map(center=(43.5, -0.5), zoom=8)

m.add_layer(Polygon(
    locations=poly.coords,
    color="green",
    fill_color="green"
))

m.add_layer(Polygon(
    locations=poly_2.coords,
    color="red",
    fill_color="red"
))

m.add_layer(Polygon(
    locations=poly_3.coords,
    color="blue",
    fill_color="blue"
))

m

In [ ]:
from django.contrib.gis.geos import Polygon, MultiPolygon

BIG_SQUARE = MultiPolygon([
    Polygon.from_bbox((43, -1, 44, 0)),
], srid=4326)

BIG_SQUARE.transform(2154, clone=True).area / 1000 ** 2